In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
import umap
import hdbscan

C:\Users\Lukasz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Wczytanie danych

In [2]:
hc_training = pd.read_csv('../../../classifier_data/HC_training.csv', sep='\t').T
hc_test = pd.read_csv('../../../classifier_data/HC_test.csv', sep='\t').T
oc_training = pd.read_csv('../../../classifier_data/OC_training.csv', sep='\t').T
oc_test = pd.read_csv('../../../classifier_data/OC_test.csv', sep='\t').T

synthetic_hc_data_path = '../../../best_models/HC/GAN/1000/synthetic_data/'
model_id = os.listdir(synthetic_hc_data_path)[0]
hc_synthetic = pd.read_csv(f'{synthetic_hc_data_path}/{model_id}/generated_data.tsv', sep='\t')

synthetic_oc_data_path = '../../../best_models/OC/GAN/1000/synthetic_data/'
model_id = os.listdir(synthetic_oc_data_path)[0]
oc_synthetic = pd.read_csv(f'{synthetic_oc_data_path}/{model_id}/generated_data.tsv', sep='\t')

# combine training and test data with labels
hc_training['label'] = 'HC'
hc_test['label'] = 'HC'
oc_training['label'] = 'OC'
oc_test['label'] = 'OC'

hc_synthetic['label'] = 'HC'
oc_synthetic['label'] = 'OC'

training_data = pd.concat([hc_training, oc_training])
test_data = pd.concat([hc_test, oc_test])

In [3]:
# keep only the columns that are in both datasets
# Znalezienie wspólnych kolumn
common_columns = training_data.columns.intersection(test_data.columns)
common_columns = common_columns.intersection(hc_synthetic.columns)
common_columns = common_columns.intersection(oc_synthetic.columns)

# Uporządkowanie kolumn w real_data
training_data = training_data[common_columns]

# Uporządkowanie kolumn w synthetic_data
test_data = test_data[common_columns]

# Uporządkowanie kolumn w synthetic_data
hc_synthetic = hc_synthetic[common_columns]
oc_synthetic = oc_synthetic[common_columns]

common_columns

Index(['ENSG00000000938', 'ENSG00000002549', 'ENSG00000004487',
       'ENSG00000004534', 'ENSG00000005059', 'ENSG00000005700',
       'ENSG00000005844', 'ENSG00000005955', 'ENSG00000006744',
       'ENSG00000007392',
       ...
       'ENSG00000243955', 'ENSG00000244038', 'ENSG00000244734',
       'ENSG00000249072', 'ENSG00000255823', 'ENSG00000257207',
       'ENSG00000266714', 'ENSG00000269028', 'ENSG00000271043', 'label'],
      dtype='object', length=716)

In [4]:
# add synthetic data to training data
training_data_synthetic = pd.concat([training_data, hc_synthetic, oc_synthetic])
training_data_synthetic

,ENSG00000000938,ENSG00000002549,ENSG00000004487,ENSG00000004534,ENSG00000005059,ENSG00000005700,ENSG00000005844,ENSG00000005955,ENSG00000006744,ENSG00000007392,...,ENSG00000243955,ENSG00000244038,ENSG00000244734,ENSG00000249072,ENSG00000255823,ENSG00000257207,ENSG00000266714,ENSG00000269028,ENSG00000271043,label
VUMC-HC-0033-TR2591,4.432779,4.954165,4.211676,4.722403,4.399111,4.328301,4.211676,4.465409,4.125114,3.793786,...,5.112603,4.954165,6.995484,6.002046,6.095028,9.847484,4.465409,7.757440,6.525607,HC
Vumc-HD-70-TR1062,5.786592,6.072522,5.309622,5.479209,5.828302,5.215895,5.247890,5.743579,5.005196,4.546459,...,5.079361,5.908122,10.021953,5.215895,5.215895,8.262227,4.139052,6.645036,5.397116,HC
VUMC-HC0053-DOT-HD-48h-TR3087,5.526849,4.724280,4.292393,4.253528,4.913257,4.170728,4.934939,4.868709,4.956248,4.126378,...,4.588670,4.822471,7.050183,3.861049,3.861049,8.831693,4.030369,4.253528,3.720811,HC
Vumc-HD-149-TR932,4.923042,4.945413,4.727528,3.808559,4.673182,4.779470,3.937986,5.275333,4.923042,3.808559,...,4.146284,4.804614,8.661739,6.247257,6.360425,3.647372,3.808559,8.517492,6.826289,HC
Vumc-HD-36-TR1165,6.484772,4.751692,5.811822,7.053716,5.998264,4.402386,5.938859,6.443015,5.020424,5.998264,...,3.862107,5.596133,9.877834,5.515912,5.241598,3.862107,6.355612,7.025844,5.339481,HC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,5.562140,4.354453,5.623399,5.130230,6.173387,4.489610,4.786326,7.862160,4.571603,4.690963,...,4.382321,6.025777,11.492651,4.334881,4.346471,7.435487,5.845482,5.200995,4.643215,OC
77,5.536098,5.623424,4.826021,4.692031,5.539174,4.231295,4.320268,6.153614,4.664353,4.132097,...,4.412487,5.729760,11.565980,4.575498,4.324905,7.389845,6.307029,5.392130,3.967898,OC
78,3.777912,4.103905,3.853299,3.664929,3.362853,4.117625,3.282083,4.083473,3.952508,3.524694,...,4.288810,4.795808,9.299460,5.794053,6.746888,7.869861,3.878739,6.313983,5.144329,OC
79,4.167622,4.114445,4.241281,4.630835,3.458137,4.228414,3.445830,4.123851,3.549432,3.493083,...,4.880588,4.465356,8.003525,6.270363,5.879812,7.477252,3.732721,4.670305,6.954122,OC


### Wykorzystanie klasyfikatorów

In [5]:
# klasyfikacja z uyciem svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


def svm_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Klasyfikacja przy użyciu SVM
    svm = SVC(kernel='linear', random_state=42)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))


svm_classification(training_data, test_data)
svm_classification(training_data_synthetic, test_data)

Dokładność: 0.9351351351351351
              precision    recall  f1-score   support

          HC       0.95      0.95      0.95       131
          OC       0.89      0.89      0.89        54

    accuracy                           0.94       185
   macro avg       0.92      0.92      0.92       185
weighted avg       0.94      0.94      0.94       185

[[125   6]
 [  6  48]]
Dokładność: 0.918918918918919
              precision    recall  f1-score   support

          HC       0.93      0.95      0.94       131
          OC       0.88      0.83      0.86        54

    accuracy                           0.92       185
   macro avg       0.91      0.89      0.90       185
weighted avg       0.92      0.92      0.92       185

[[125   6]
 [  9  45]]


In [6]:
from sklearn.tree import DecisionTreeClassifier
# klasyfikacja z uyciem random forest
def decision_tree_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    dt = DecisionTreeClassifier(max_depth=5, random_state=42)
    dt.fit(X_train, y_train)
    y_pred = dt.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

decision_tree_classification(training_data, test_data)
decision_tree_classification(training_data_synthetic, test_data)

Dokładność: 0.8
              precision    recall  f1-score   support

          HC       0.83      0.90      0.86       131
          OC       0.70      0.56      0.62        54

    accuracy                           0.80       185
   macro avg       0.76      0.73      0.74       185
weighted avg       0.79      0.80      0.79       185

[[118  13]
 [ 24  30]]
Dokładność: 0.8324324324324325
              precision    recall  f1-score   support

          HC       0.85      0.92      0.89       131
          OC       0.77      0.61      0.68        54

    accuracy                           0.83       185
   macro avg       0.81      0.77      0.78       185
weighted avg       0.83      0.83      0.83       185

[[121  10]
 [ 21  33]]


In [7]:
from sklearn.ensemble import RandomForestClassifier
# klasyfikacja z uyciem random forest
def random_forest_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

random_forest_classification(training_data, test_data)
random_forest_classification(training_data_synthetic, test_data)

Dokładność: 0.8486486486486486
              precision    recall  f1-score   support

          HC       0.82      1.00      0.90       131
          OC       1.00      0.48      0.65        54

    accuracy                           0.85       185
   macro avg       0.91      0.74      0.78       185
weighted avg       0.88      0.85      0.83       185

[[131   0]
 [ 28  26]]
Dokładność: 0.8540540540540541
              precision    recall  f1-score   support

          HC       0.84      0.98      0.91       131
          OC       0.94      0.54      0.68        54

    accuracy                           0.85       185
   macro avg       0.89      0.76      0.79       185
weighted avg       0.87      0.85      0.84       185

[[129   2]
 [ 25  29]]


In [8]:
from xgboost import XGBClassifier

# klasyfikacja z uyciem xgboost
def xgboost_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    y_train = y_train.replace('HC', 0)
    y_train = y_train.replace('OC', 1)
    y_test = y_test.replace('HC', 0)
    y_test = y_test.replace('OC', 1)
    xgb = XGBClassifier(random_state=42)
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

xgboost_classification(training_data, test_data)
xgboost_classification(training_data_synthetic, test_data)

Dokładność: 0.8810810810810811
              precision    recall  f1-score   support

           0       0.88      0.96      0.92       131
           1       0.88      0.69      0.77        54

    accuracy                           0.88       185
   macro avg       0.88      0.82      0.85       185
weighted avg       0.88      0.88      0.88       185

[[126   5]
 [ 17  37]]
Dokładność: 0.9081081081081082
              precision    recall  f1-score   support

           0       0.91      0.96      0.94       131
           1       0.89      0.78      0.83        54

    accuracy                           0.91       185
   macro avg       0.90      0.87      0.88       185
weighted avg       0.91      0.91      0.91       185

[[126   5]
 [ 12  42]]


In [9]:
from sklearn.neural_network import MLPClassifier

# klasyfikacja z uzyciem sieci neuronowych

def nn_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    nn = MLPClassifier(hidden_layer_sizes=(100,100), random_state=42)
    nn.fit(X_train, y_train)
    y_pred = nn.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

nn_classification(training_data, test_data)
nn_classification(training_data_synthetic, test_data)

Dokładność: 0.9027027027027027
              precision    recall  f1-score   support

          HC       0.93      0.93      0.93       131
          OC       0.83      0.83      0.83        54

    accuracy                           0.90       185
   macro avg       0.88      0.88      0.88       185
weighted avg       0.90      0.90      0.90       185

[[122   9]
 [  9  45]]
Dokładność: 0.9081081081081082
              precision    recall  f1-score   support

          HC       0.93      0.94      0.94       131
          OC       0.85      0.83      0.84        54

    accuracy                           0.91       185
   macro avg       0.89      0.89      0.89       185
weighted avg       0.91      0.91      0.91       185

[[123   8]
 [  9  45]]
